In [1]:
#!pip install bayesian-optimization
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.model_selection import KFold
import time

from lightgbm import LGBMClassifier
import lightgbm as lgb

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
%matplotlib inline

import warnings
warnings.simplefilter('ignore', UserWarning)

import gc
gc.enable()

from bayes_opt import BayesianOptimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=d231135744d3b5c9e24b39eed9fa8cb0c4d57389ff4f3de37617b413012d7c28
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
train = pd.read_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/train2.csv')
test = pd.read_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/test2.csv')

In [4]:
print('train dataset has {} rows and {} columns.'.format(train.shape[0],train.shape[1]))
print('test dataset has {} rows and {} columns.'.format(test.shape[0],test.shape[1]))

train dataset has 1754884 rows and 124 columns.
test dataset has 113640 rows and 122 columns.


In [5]:
train = train[train['label']!=-1]

In [6]:
train

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,discount,weekday,day,u2,u3,u19,u1,u4,u5,u25,u20,u6,u26,u7,u8,u9,u10,u11,u21,u22,u23,u24,u45,u27,u28,u32,u47,u33,u34,u35,u36,u37,...,um4,um5,um6,um7,um8,um9,um10,um11,um12,o1,o2,o3,o4,o5,o6,o7,o8,o9,o10,o11,o12,o13,o14,o15,o16,o17,o18,on_u1,on_u2,on_u3,on_u4,on_u5,on_u6,on_u7,on_u9,on_u8,on_u10,on_u11,on_u12,on_u13
1,1439408,4663,11002.0,150:20,1.0,2016-05-28,NaN,0,0.866667,5.0,28.0,1.0,4.0,0.25,5.0,0.2,2.0,3.0,0.333333,58.50,89.0,NaN,3.900000,NaN,28.00,1.866667,NaN,NaN,NaN,NaN,0.95,0.950000,0.950000,1.0,1.0,1.000000,1.000000,0.0,0.0,0.0,...,NaN,0.25,NaN,NaN,1.0,2.0,1.0,0.500000,0.500000,7,1.0,3,1,1,3,19.500000,1,2,1.0,1.0,3.0,16549,7730.0,15079,6.0,12.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
2,1439408,2632,8591.0,20:1,0.0,2016-02-17,NaN,0,0.950000,2.0,17.0,1.0,4.0,0.25,5.0,0.2,2.0,3.0,0.333333,58.50,89.0,NaN,3.900000,NaN,28.00,1.866667,NaN,NaN,NaN,NaN,0.95,0.950000,0.950000,1.0,3.0,0.333333,0.333333,0.0,0.0,0.0,...,0.25,0.75,3.0,2.0,1.0,2.0,1.0,0.500000,0.500000,7,3.0,0,4,2,2,19.500000,6,2,1.0,1.0,3.0,60,31.0,15,2.0,NaN,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
3,1439408,2632,1078.0,20:1,0.0,2016-03-19,NaN,0,0.950000,5.0,19.0,1.0,4.0,0.25,5.0,0.2,2.0,3.0,0.333333,58.50,89.0,NaN,3.900000,NaN,28.00,1.866667,NaN,NaN,NaN,NaN,0.95,0.950000,0.950000,1.0,1.0,1.000000,1.000000,0.0,0.0,0.0,...,0.25,0.75,3.0,2.0,1.0,2.0,1.0,0.500000,0.500000,7,1.0,1,3,0,4,19.500000,6,2,1.0,1.0,3.0,60,12.0,15,2.0,31.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
4,1439408,2632,8591.0,20:1,0.0,2016-06-13,NaN,0,0.950000,0.0,13.0,1.0,4.0,0.25,5.0,0.2,2.0,3.0,0.333333,58.50,89.0,NaN,3.900000,NaN,28.00,1.866667,NaN,NaN,NaN,NaN,0.95,0.950000,0.950000,1.0,3.0,0.333333,0.333333,0.0,0.0,0.0,...,0.25,0.75,3.0,2.0,1.0,2.0,1.0,0.500000,0.500000,7,3.0,4,0,4,0,19.500000,6,2,1.0,1.0,3.0,60,31.0,15,2.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
6,1439408,2632,8591.0,20:1,0.0,2016-05-16,2016-06-13,0,0.950000,0.0,16.0,1.0,4.0,0.25,5.0,0.2,2.0,3.0,0.333333,58.50,89.0,NaN,3.900000,NaN,28.00,1.866667,NaN,NaN,NaN,NaN,0.95,0.950000,0.950000,1.0,3.0,0.333333,0.333333,0.0,0.0,0.0,...,0.25,0.75,3.0,2.0,1.0,2.0,1.0,0.500000,0.500000,7,3.0,2,2,3,1,19.500000,6,2,1.0,1.0,3.0,60,31.0,15,2.0,58.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1754877,212662,3021,3739.0,30:1,6.0,2016-05-04,2016-05-08,1,0.966667,2.0,4.0,4.0,1.0,4.00,5.0,0.8,5.0,9.0,0.444444,10.75,16.5,24.0,0.716667,1.6,9.75,0.650000,3.0,0.75,3.0,0.6,0.90,0.833333,0.966667,2.0,2.0,1.000000,2.000000,4.0,2.0,6.0,...,1.00,NaN,2.0,NaN,1.0,3.0,2.0,0.666667,1.333333,10,2.0,3,1,0,4,5.333333,2,3,1.0,1.0,3.0,19,11.0,10,1.0,43.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
1754878,212662,2934,5686.0,30:5,2.0,2016-03-21,2016-03-22,1,0.833333,0.0,21.0,4.0,1.0,4.00,5.0,0.8,5.0,9.0,0.444444,10.75,16.5,24.0,0.716667,1.6,9.75,0.650000,3.0,0.75,3.0,0.6,0.90,0.833333,0.966667,2.0,2.0,1.000000,2.000000,4.0,2.0,6.0,...,1.00,NaN,5.0,3.0,2.0,3.0,2.0,0.666667,1.333333,10,2.0,1,4,4,1,5.333333,5,3,2.0,2.0,3.0,40645,10533.0,7227,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
1754880,212662,3021,3739.0,30:1,6.0,2016-05-08,2016-06-02,0,0.966667,6.0,8.0,4.0,1.0,4.00,5.0,0.8,5.0,9.0,0.444444,10.75,16.5,24.0,0.716667,1.6,9.75,0.650000,3.0,0.75,3.0,0.6,0.90,0.833333,0.966667,2.0,2.0,1.000000,2.000000,4.0,2.0,6.0,...,1.00,NaN,2.0,NaN,1.0,3.0,2.0,0.666667,1.333333,10,2.0,4,0,1,3,5.333333,2,3,1.0,1.0,3.0,19,11.0,10,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
1754882,752472,7113,1633.0,50:10,6.0,2016-06-13,NaN,0,0.800000,0.0,13.0,NaN,2.0,N

# Feature Selection

In [7]:
categorical_feats = ['weekday','day']

In [8]:
def get_feature_importances(data, shuffle, seed=None):
    # Gather real features
    train_features = [f for f in data if f not in ['User_id','Merchant_id','Coupon_id','Date_received','Discount_rate','Date','label']]
    # Go over fold and keep track of CV score (train and valid) and feature importances
    
    # Shuffle target if required
    y = data['label'].copy()
    if shuffle:
        # Here you could as well use a binomial distribution
        y = data['label'].copy().sample(frac=1.0)
    
    # Fit LightGBM in RF mode, yes it's quicker than sklearn RandomForest
    dtrain = lgb.Dataset(data[train_features], y, free_raw_data=False, silent=True)
    lgb_params = {
        'objective': 'binary',
        'boosting_type': 'rf',
        'subsample': 0.623,
        'colsample_bytree': 0.7,
        'num_leaves': 127,
        'max_depth': 8,
        'seed': seed,
        'bagging_freq': 1,
        'n_jobs': 4
    }
    
    # Fit the model
    clf = lgb.train(params=lgb_params, train_set=dtrain, num_boost_round=200, categorical_feature=categorical_feats)

    # Get feature importances
    imp_df = pd.DataFrame()
    imp_df["feature"] = list(train_features)
    imp_df["importance_gain"] = clf.feature_importance(importance_type='gain')
    imp_df["importance_split"] = clf.feature_importance(importance_type='split')
    imp_df['trn_score'] = roc_auc_score(y, clf.predict(data[train_features]))
    
    
    return imp_df

True Importance

In [ ]:
# Seed the unexpected randomness of this world
np.random.seed(123)
# Get the actual importance, i.e. without shuffling
actual_imp_df = get_feature_importances(data=train, shuffle=False)

In [ ]:
actual_imp_df.sort_values('importance_split')

,feature,importance_gain,importance_split,trn_score
2,weekday,0.000000e+00,0,0.998313
111,on_u9,0.000000e+00,0,0.998313
115,on_u12,0.000000e+00,0,0.998313
61,m22,0.000000e+00,0,0.998313
3,day,2.109400e+01,1,0.998313
...,...,...,...,...
77,um4,6.269406e+07,1107,0.998313
102,o17,7.966274e+05,1285,0.998313
103,o18,1.812913e+06,1526,0.998313
20,u22,6.913325e+06,1555,0.998313


Null Importance

In [ ]:
null_imp_df = pd.DataFrame()
nb_runs = 80
import time
start = time.time()
dsp = ''
for i in range(nb_runs):
    # Get current run importances
    imp_df = get_feature_importances(data=train, shuffle=True)
    imp_df['run'] = i + 1 
    # Concat the latest importances with the old ones
    null_imp_df = pd.concat([null_imp_df, imp_df], axis=0)
    # Erase previous message
    for l in range(len(dsp)):
        print('\b', end='', flush=True)
    # Display current run and time used
    spent = (time.time() - start) / 60
    dsp = 'Done with %4d of %4d (Spent %5.1f min)' % (i + 1, nb_runs, spent)
    print(dsp, end='', flush=True)

Done with    1 of   80 (Spent   0.9 min)Done with    2 of   80 (Spent   1.8 min)Done with    3 of   80 (Spent   2.8 min)Done with    4 of   80 (Spent   3.8 min)

KeyboardInterrupt: ignored

In [ ]:
null_imp_df.head()

In [ ]:
feature_scores = []
for _f in actual_imp_df['feature'].unique():
    f_null_imps_gain = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_gain'].values
    f_act_imps_gain = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_gain'].mean()
    gain_score = np.log(1e-10 + f_act_imps_gain / (1 + np.percentile(f_null_imps_gain, 75)))  # Avoid didvide by zero
    f_null_imps_split = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_split'].values
    f_act_imps_split = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_split'].mean()
    split_score = np.log(1e-10 + f_act_imps_split / (1 + np.percentile(f_null_imps_split, 75)))  # Avoid didvide by zero
    feature_scores.append((_f, split_score, gain_score))

scores_df = pd.DataFrame(feature_scores, columns=['feature', 'split_score', 'gain_score'])

plt.figure(figsize=(16, 16))
gs = gridspec.GridSpec(1, 2)
# Plot Split importances
ax = plt.subplot(gs[0, 0])
sns.barplot(x='split_score', y='feature', data=scores_df.sort_values('split_score', ascending=False).iloc[0:70], ax=ax)
ax.set_title('Feature scores wrt split importances', fontweight='bold', fontsize=14)
# Plot Gain importances
ax = plt.subplot(gs[0, 1])
sns.barplot(x='gain_score', y='feature', data=scores_df.sort_values('gain_score', ascending=False).iloc[0:70], ax=ax)
ax.set_title('Feature scores wrt gain importances', fontweight='bold', fontsize=14)
plt.tight_layout()


In [ ]:
correlation_scores = []
for _f in actual_imp_df['feature'].unique():
    f_null_imps = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_gain'].values
    f_act_imps = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_gain'].values
    gain_score = 100 * (f_null_imps < np.percentile(f_act_imps, 25)).sum() / f_null_imps.size
    f_null_imps = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_split'].values
    f_act_imps = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_split'].values
    split_score = 100 * (f_null_imps < np.percentile(f_act_imps, 25)).sum() / f_null_imps.size
    correlation_scores.append((_f, split_score, gain_score))

corr_scores_df = pd.DataFrame(correlation_scores, columns=['feature', 'split_score', 'gain_score'])

fig = plt.figure(figsize=(16, 16))
gs = gridspec.GridSpec(1, 2)
# Plot Split importances
ax = plt.subplot(gs[0, 0])
sns.barplot(x='split_score', y='feature', data=corr_scores_df.sort_values('split_score', ascending=False).iloc[0:70], ax=ax)
ax.set_title('Feature scores wrt split importances', fontweight='bold', fontsize=14)
# Plot Gain importances
ax = plt.subplot(gs[0, 1])
sns.barplot(x='gain_score', y='feature', data=corr_scores_df.sort_values('gain_score', ascending=False).iloc[0:70], ax=ax)
ax.set_title('Feature scores wrt gain importances', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.suptitle("Features' split and gain scores", fontweight='bold', fontsize=16)
fig.subplots_adjust(top=0.93)

Check Removal Threshold

In [ ]:
def score_feature_selection(df=None, train_features=None, cat_feats=None, target=None):
    # Fit LightGBM 
    dtrain = lgb.Dataset(df[train_features], target, free_raw_data=False, silent=True)
    lgb_params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'learning_rate': .1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'num_leaves': 31,
        'max_depth': -1,
        'seed': 13,
        'n_jobs': 4,
        'min_split_gain': .00001,
        'reg_alpha': .00001,
        'reg_lambda': .00001,
        'metric': 'auc'
    }
    
    # Fit the model
    hist = lgb.cv(
        params=lgb_params, 
        train_set=dtrain, 
        num_boost_round=2000,
        categorical_feature=cat_feats,
        nfold=5,
        stratified=True,
        shuffle=True,
        early_stopping_rounds=50,
        verbose_eval=0,
        seed=17
    )
    # Return the last mean / std values 
    return hist['auc-mean'][-1], hist['auc-stdv'][-1]

# features = [f for f in data.columns if f not in ['SK_ID_CURR', 'TARGET']]
# score_feature_selection(df=data[features], train_features=features, target=data['TARGET'])

for threshold in [0, 10, 20, 30 , 40, 50 ,60 , 70, 80 , 90, 95, 99]:
    split_feats = [_f for _f, _score, _ in correlation_scores if _score >= threshold]
    split_cat_feats = [_f for _f, _score, _ in correlation_scores if (_score >= threshold) & (_f in categorical_feats)]
    gain_feats = [_f for _f, _, _score in correlation_scores if _score >= threshold]
    gain_cat_feats = [_f for _f, _, _score in correlation_scores if (_score >= threshold) & (_f in categorical_feats)]
                                                                                             
    print('Results for threshold %3d' % threshold)
    split_results = score_feature_selection(df=train, train_features=split_feats, cat_feats=split_cat_feats, target=train['label'])
    print('There are {} features remained.'.format(len(split_results)))
    print('\t SPLIT : %.6f +/- %.6f' % (split_results[0], split_results[1]))
    gain_results = score_feature_selection(df=train, train_features=gain_feats, cat_feats=gain_cat_feats, target=train['label'])
    print('There are {} features remained.'.format(len(gain_feats)))
    print('\t GAIN  : %.6f +/- %.6f' % (gain_results[0], gain_results[1]))

# Bayesian Optimization

In [9]:
features = [x for x in train.columns if x not in ['User_id','Merchant_id','Coupon_id','Date_received','Discount_rate','Date','label']]
#features = gain_feats
X = train[features]
y = train['label']
X_test = test[features]
del train, test

categorical_feats = []

In [ ]:
def bayes_parameter_opt_lgb(X,y,init_round=15,opt_round=25,n_folds=5,random_seed=0,
              n_estimators=10000,learning_rate=0.02,output_process=False):
  # Prepare data
  train_data = lgb.Dataset(data=X,label=y,free_raw_data=False)

  # Parameters
  def lgb_eval(num_leaves,max_depth,lambda_l1,lambda_l2,min_split_gain,min_child_weight):
    params={'application':'binary','num_iterations':n_estimators,'learning_rate':learning_rate,
            'early_stopping_round':200,'metric':'roc'}
    params['nthread'] = 4
    params['num_leaves'] = int(round(num_leaves))
    params['max_depth'] = int(round(max_depth))
    params['lambda_l1'] = max(lambda_l1,0)
    params['lambda_l2'] = max(lambda_l2,0)
    params['min_split_gain'] = min_split_gain
    params['min_child_weight'] = min_child_weight
    params['silent'] = -1
    params['verbose'] = -1
    cv_result=lgb.cv(params,train_data,nfold=n_folds,seed=random_seed,verbose_eval=200,
                     metrics=['auc'],categorical_feature=categorical_feats)
    return max(cv_result['auc-mean'])
    
  # Range
  lgbBO = BayesianOptimization(lgb_eval,{'num_leaves':(24,45),
                        'max_depth':(5,45),
                        'lambda_l1':(0,5),
                        'lambda_l2':(0,3),
                        'min_split_gain':(0.001,0.1),
                        'min_child_weight':(5,50)},random_state=0)
  # Optimize
  lgbBO.maximize(init_points=init_round,n_iter=opt_round)

  # Output optimization process
  #if output_process == True: lgbBO.points_to_csv('/content/drive/My Drive/Case/Repeat Buyers Prediction/data/bayes_opt_result.csv')
  # Return best parameters
  return lgbBO.max['params']

lgb_opt_params = bayes_parameter_opt_lgb(X,y,init_round=5,opt_round=10,n_folds=5,random_seed=0,
                                  n_estimators=10000,learning_rate=0.02)
lgb_opt_params

# LGBM on KFold

In [16]:
params = {
    'boost': 'gbdt',
    'learning_rate': 0.02,
    'max_depth': 20,
    'metric':'auc',
    'num_leaves': 25,
    'num_threads': 4,
    'objective': 'rank:pairwise'
}

In [17]:
num_folds = 5

#features = gain_feats

folds = KFold(n_splits=num_folds, random_state=1234)
oof = np.zeros(len(X))
getVal = np.zeros(len(X))
predictions = np.zeros(len(X_test))
feature_importance_df = pd.DataFrame()

#params={'application':'binary','num_iterations':10000,'learning_rate':0.02,
#            'metric':'roc'}
#params['nthread'] = 4
#params['num_leaves'] = int(round(lgb_opt_params['num_leaves']))
#params['max_depth'] = int(round(lgb_opt_params['max_depth']))
#params['lambda_l1'] = max(lgb_opt_params['lambda_l1'],0)
#params['lambda_l2'] = max(lgb_opt_params['lambda_l2'],0)
#params['min_split_gain'] = lgb_opt_params['min_split_gain']
#params['min_child_weight'] = lgb_opt_params['min_child_weight']
#params['silent'] = -1
#params['verbose'] = -1

print('Light GBM Model')
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X.values, y.values)):
    
    X_train, y_train = X.iloc[trn_idx][features], y.iloc[trn_idx]
    X_valid, y_valid = X.iloc[val_idx][features], y.iloc[val_idx]
    
    #X_tr, y_tr = augment(X_train.values, y_train.values)
    #X_tr = pd.DataFrame(X_tr)
    
    print("Fold idx:{}".format(fold_ + 1))
    trn_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_valid, label=y_valid)
    
    clf = lgb.train(params, trn_data, 10000, valid_sets = [trn_data, val_data],
                    early_stopping_rounds=100, verbose_eval=1000,
                    categorical_feature=categorical_feats)
    oof[val_idx] = clf.predict(X.iloc[val_idx][features], num_iteration=clf.best_iteration)
    getVal[val_idx]+= clf.predict(X.iloc[val_idx][features], num_iteration=clf.best_iteration) / folds.n_splits
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(X_test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(roc_auc_score(y, oof)))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Light GBM Model
Fold idx:1


LightGBMError: ignored

# 输出结果

In [12]:
submission = pd.read_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/ccf_offline_stage1_test_revised.csv')


In [13]:
submission['Probability'] = predictions
submission = submission[['User_id','Coupon_id','Date_received','Probability']]

In [14]:
submission

,User_id,Coupon_id,Date_received,Probability
0,4129537,9983,20160712,4.615054e-11
1,6949378,3429,20160706,5.659417e-04
2,2166529,6928,20160727,1.132200e-09
3,2166529,1808,20160727,8.356706e-10
4,6172162,6500,20160708,7.346414e-10
...,...,...,...,...
113635,5828093,10418,20160716,1.103722e-10
113636,6626813,7595,20160707,8.291312e-09
113637,6626813,7590,20160712,6.183437e-08
113638,4547069,13602,20160717,8.741665e-09


In [15]:
submission.to_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/Mysubmission2.csv', index=0)